Этот блокнот - быстрая демонстрация того, что было 

# SETUP

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import re

import pymorphy2
import nltk
from tqdm import tqdm

import pickle

import sys
def add_sys_path(p):
    if p not in sys.path:
        sys.path.append(p)
add_sys_path(os.path.dirname(os.getcwd()))

from test_problem import util


Вот этот модуль собрал внутри себя всю мою вечернюю, ночную и утреннюю боль

In [2]:
from synthetic.naive import apply_to_new_data

In [3]:
INPUT_ROOT = os.path.join('..','INPUT')

загружаю первый кусок необходимых данных: word2vec

In [4]:
w2v_fpath = os.path.join(r"C:\Users\ddale\Downloads\mv", "all.norm-sz100-w10-cb0-it1-min100.w2v")
import gensim
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v.init_sims(replace=True)
vocab = set(w2v.index2word)

C:\Users\ddale\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Загружаю счётчики слов

In [5]:
with open('total_counter.pkl', 'rb') as file:
    total_counter = pickle.load(file)
with open('total_df.pkl', 'rb') as file:
    total_df = pickle.load(file)

# Применение 

In [6]:
data = pd.read_csv('proba_1_v5.csv', index_col=0)
print(data.shape)
data.head(3)

(1539, 16)


,PRODUCT,NAME,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,20025017,Морозильная камера Liebherr G 1213-20 ...,4020301,313,FREEZERS,Liebherr,review,NaN,"Небольшая симпатичная морозильная камера, нашл...",02.09.2013,4.0,NaN,NaN,0,30,26
1,20025017,Морозильная камера Liebherr G 1213-20 ...,4020301,313,FREEZERS,Liebherr,review,NaN,Пользуемся месяц. Искали недорогую маленькую м...,17.09.2017,5.0,низкая цена по сравнению с другими с такими же...,Пока не нашла,0,1,0
2,20025017,Морозильная камера Liebherr G 1213-20 ...,4020301,313,FREEZERS,Liebherr,review,NaN,"Морозилка хорошая, подошла по высоте кухонного...",25.08.2017,4.7,NaN,NaN,0,1,0


In [7]:
data.PRODUCT.value_counts()

20035772    254
20036154    152
30024248    103
30026637     97
20030060     93
10006402     89
20029099     88
20036294     78
20027607     59
20025017     56
30026139     54
20038567     53
10010083     50
20029414     50
20029680     47
30022055     43
20036717     42
30027840     36
30025963     30
10008332     30
10012105      8
20039126      8
30029823      8
30028898      7
30028579      4
Name: PRODUCT, dtype: int64

Найдём кластеры для одного продукта. 

In [8]:
prod1 = data[data.PRODUCT==30025963].copy()
result = apply_to_new_data(prod1, total_counter, total_df, w2v, vocab)

[-1  0  0  0 -1  0 -1 -1  1  1 -1 -1  0  0  0 -1 -1  0  0  2  2 -1 -1 -1 -1
 -1 -1 -1 -1 -1  0  0 -1 -1 -1  0  0  0 -1  0  0  0  0  0  0 -1 -1  0  0 -1
 -1  0 -1  3 -1 -1  4 -1 -1 -1 -1  0  5  6 -1 -1 -1 -1 -1 -1 -1  0  0 -1 -1
  7  8 -1 -1 -1  0  0 -1 -1  9  9 -1  0  0  0  0  0  0  4 -1 -1 10  0 -1 11
 -1 12  6 -1 13 -1 14 14 10 15 15 -1 -1 -1 16 16 16 -1 17 -1 18 -1 18 -1 -1
 -1 -1 -1 19 11 20 -1  0  0  0  0  0  0  0 21 -1 -1 -1 -1 17 -1  0 -1 -1 -1
  0 -1 -1 -1 -1 -1 -1 -1 22 -1 22  0  0  7 -1 -1 -1 13 -1 -1 -1  0 -1 -1 -1
 -1 -1 -1 -1  5 23 -1 -1  6 -1 24  5 -1 -1 -1 25 -1 26 -1 -1 -1 -1 26  0 -1
 24 -1  0  0 -1  4 -1 -1 27  0 -1 -1 28 28 29 29 29  0 -1 -1 30 30 -1 -1  0
 -1 -1 -1 31 -1 31  0 17  0  0 -1  4  0 32 32 -1 -1 -1 20 -1  3 -1 -1  0  0
 -1 -1  0 24  0 -1 -1  0 -1  0 -1  0 33 33 -1  0  0 -1 -1  0  0 34 -1 -1 -1
 -1  0  0  0  0  0  0  0  0 -1  8 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1 -1 12 -1
 23  6 35 35 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 36 36 -1 -1 -1 -1 -1
  0  0  0  0

Вот что выдаёт в итоге моя модель. Грамматики маловато. 

In [9]:
for k, v in result.items():
    print(k)
    print(v)
    print()

Ноутбук ASUS X541UV-XO241T                                           
Пользователи пишут:
 > Проблем благодаря опыту пользования windows 10 в течение пары лет не было
 > Жёсткий диск-крехтит
 > Через 3 недели полетел жесткий диск
 > Сильно не греется и не шумит
 > Что ноутбук относительно лёгкий и тонкий
 > Если поменять жесткий диск
 > Отличный ноутбук
 > Покупала для работы и учебы
 > Производительный
 > Пока претензий нет
 > Хороший процессор
 > Не советую гг
 > Видеокарта
 > Процессор
 > Дизайн
 > Качество
 > Сборка
 > Хороший
 > В целом
 > Экран
 > Звук



А вот что пишут на самом деле:

In [10]:
for x in prod1.total_text.values:
    print(x)
    print('------------------')


***
Достаточно мощный процессор, не смотрите на цифру 6198, это тот же 6200 только с чуть худшим графическим ядром, да и на кой оно нужно, если стоит, в общем-то, современная дискретная графика. Перечитал кучу отзывов за 3 дня, сначала остановился на модели Acer  с GF 950mx и проц (по отзывам) I5 7200U, но так как остался только один и то с витрины, решил обратиться к ASUS. До этого было 2 ноутбука ASUS 2008 года, и ACER 2013го. На ASUSе через год полетел привод, на ACERе -  HDD через 3 года, оба работают до сих пор. Насторожили отзывы по данной модели по HDD, и щелчки действительно подозрительны, но пока работает, подкоплю пока на SSD. Покупал для..., ну просто захотелось ноутбук, 2 предыдущих отдал матери для работы, определенных требований не было, просто хотелось первый компьютер с i5 на борту(до этого были не лучше i3), чтоб хорошо тянул Photoshop, ну более или менее приличной графикой, чтобы иногда закинуться GTA V, и в общем-то оправдал ожидания процентов на 110. Было бы 150, н

качество сборки
------------------

***
Приобрел сей дейвайс, впечатления - ужасно! Не соответсвует цене. Играю в простые игры троит и видеокартка начинает борохлить... После перезагрузки все меняется . Повторяется раз в неделю прмерно. Не рекомендую
***

***

------------------

***
Ноутбук asus всегда со мной, потому что я постоянно нахожусь в движении и работаю в любом месте и в любое время.  Модель очень легкая и компактная, поэтому ее удобно носить в сумке. Ноутбук смотрится очень эффектно, и не стыдно достать на деловых встречах)) Работаю в MS Office, photoshop, coral , все работает быстро   ( в данной модели двухъядерный процессор  Core i5 и 8 ГБ оперативной памяти).  Качество изображения высочайшее. Все четкое и красочное, благодаря мощной видеокарте последнего поколения GeForce 920MX 2ГБ.  Характеристики данной модели превосходные и позволяют играть в любые игры , работать в любых программах и приложениях без нареканий. В свободное от работы время смотрю на ноутбуке фильмы раз

Что пишут в тайтлах:

In [13]:
import re

In [15]:
??re.sub

In [19]:
def join_texts(series):
    return re.sub('(\n)+', '\n', '\n'.join(series.fillna('').values))
print(join_texts(prod1.SUBJECT))

In [20]:
print(join_texts(prod1.BENEFITS))

Хороший процессор, достойная дискретная графика, много оперативки, сильно не греется и не шумит
Дизайн
Хороший, производительный, мощный!!!
Охлаждение, процессор, стиль, звук.
Не знаю
Процессор, видеокарта, звуковая система - это всё на высоте
крутой звук удобный лёгкий красивый хорошая клавиатура
2 видеокарты(Intel и NVIDIA). Проц 2-х ядерный-логический. Аккумулятор. Дизайн. Сборка. Железо оригинал(не Китай)
Дизайн, функциональность, батарея
нет
Хорошее изображение,отличный звук.
Качество экрана, оперативная память, видеокарта, собственные динамики, неплохой процессор
лёгкий стильный
качественная сборка, производительный
производительность, клавиатура, звук
легкость,
Эксперт октября Простота, качество, лучшая цена по сравнению с другими производителями
Покупали сыну для учёбы,ездить с ним и дома работать. Прекрасный аппарат,в пользовании около месяца,проблем пока никаких нет. Маловато USB портов (2 штуки),выручает недавно купленный Хаб,зато ноутбук лёгкий и мощный,рекомендую по соотно

In [21]:
print(join_texts(prod1.DRAWBACKS))

2 USB порта, ну граждане, даже у модели дешевле их 3! Всего один разъем под оперативку.
Жёсткий диск-крехтит,щелкает и т.д. очень сильно,кривой корпус (выпеклый в правой части клавиатуры),испорчен правый динамик.
Не нашел
ЖЕСтКИЙ ДИСК, камера, экран.
Не знаю
Экран - обычный TFT 1366х768, не то чтобы минус, но за эти деньги можно было поставить и что-то поинтереснее
пришлось обновлять биус в сервисном центре сделали это без проблем быстро
Жесткий диск абсолютное фуфло. Ставлю SSD.
Не выявил
полно
Пока претензий нет.
не нравится цвет ноутбука
при подключении зарядного устройства зависает намертво понесу в сервис
проблемы с USB и Wi-Fi
цена, сборка, жесткий диск
качество сборки
батарея рассчитана 4-5 часов работы - всё таки не хватает!
В силу тонкости немного хлипковат корпус,но это для кого как,нас устраивает.



# Отдельные элементы

In [28]:
??w2v.most_similar

In [29]:
w2v.most_similar(negative=['твердость', 'тяжесть', 'гладкость'], positive=['легкость', 'твердый', 'тяжелый', 'гладкий'])

[('неуклюжий', 0.6632130146026611),
 ('открытый', 0.6591335535049438),
 ('самый', 0.6584378480911255),
 ('длинный', 0.6565474271774292),
 ('маленький', 0.650662899017334),
 ('огромный', 0.6473284959793091),
 ('круглый', 0.6358441114425659),
 ('большущий', 0.6352936029434204),
 ('неудобный', 0.6338245868682861),
 ('обычный', 0.6329610347747803)]